### 1. Install and Import Dependencies

In [2]:
!pip install mediapipe opencv-python
!pip install ultralytics
!pip install pybboxes

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip


In [2]:
import mediapipe as mp
import cv2
import numpy as np
from ultralytics import YOLO
import mediapipe as mp
import os
from ultralytics.utils.plotting import Annotator 


### 2. Load YOLO stuff

In [3]:
weights_loc = "/Users/taylor/runs/detect/train14/weights/best.pt"
yolo_model = YOLO(weights_loc)

### 3. Hand poses

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [22]:
padding = 30
img_counter = 0
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [21]:
cap = cv2.VideoCapture(1) 

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  

        #image loading
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        results = hands.process(image)
        canvas_for_yolo = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, c = frame.shape

        #for drawing
        canvas = np.zeros_like(image)

        #draw results on black canvas
        if results.multi_hand_landmarks:
            for handLMs in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    canvas, handLMs, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)
                )

        width = canvas_for_yolo.shape[1]

        #perform YOLO predictions
        yolo_results = yolo_model.predict(frame)
        for r in yolo_results:
            #annotate boxes
            annotator = Annotator(canvas)
            boxes = r.boxes
            for box in boxes:
                b = box.xyxy[0]
                mirrored_x_min = width - b[2]
                mirrored_x_max = width - b[0]
                mirrored_box = [mirrored_x_min-padding, b[1]-padding, mirrored_x_max+padding, b[3]+padding]
                c = box.cls
                annotator.box_label(mirrored_box, "hand")
                
                #create cropped canvas for saving
                save_me = canvas[max(int(mirrored_box[1]), 0):min(int(mirrored_box[3]), canvas.shape[0]),
                               max(int(mirrored_box[0]), 0):min(int(mirrored_box[2]), canvas.shape[1])]
            
                if cv2.waitKey(10) & 0xFF == ord('s'):
                    img_name = f"cropped_hand_{img_counter}.png"
                    cv2.imwrite("skeleton_images/" + img_name, save_me)
                    print(f"{img_name} saved.")
                    img_counter += 1

        #display
        cv2.imshow('Hand Skeleton', canvas)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 156.9ms
Speed: 2.1ms preprocess, 156.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 myleft, 1 myright, 119.4ms
Speed: 4.2ms preprocess, 119.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 myright, 95.1ms
Speed: 1.5ms preprocess, 95.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 91.4ms
Speed: 1.4ms preprocess, 91.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 myright, 96.0ms
Speed: 1.6ms preprocess, 96.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 myright, 117.4ms
Speed: 1.4ms preprocess, 117.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 myright, 119.6ms
Speed: 1.4ms preprocess, 119.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 124.4ms
Speed: 1.6ms preprocess, 124.4ms inference, 